# Cyclone Nanmadol



In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import ccic
import xarray as xr

## Define area and create surface data

In [ ]:
from pathlib import Path
cpcir_files = sorted(list(Path("/home/simonpf/data_3/ccic/results/nanmadol/").glob("*.nc")))
cpcir_data = xr.open_mfdataset(cpcir_files)

In [ ]:
cpcir_data = xr.load_dataset("/home/simonpf/data_3/ccic/results/nanmadol/ccic_cpcir_202209171200.nc")

In [ ]:
cpcir_data = cpcir_data[{"latitude": slice(0, -100)}]

In [ ]:
lon_min = cpcir_data.longitude.data.min()
lon_max = cpcir_data.longitude.data.max()
lat_min = cpcir_data.latitude.data.min()
lat_max = cpcir_data.latitude.data.max()

In [ ]:
from pyresample import create_area_def
area = create_area_def(
    "pacific north-west",
    {"proj": "longlat", "datum": "WGS84"},
    area_extent=[lon_min, lat_min, lon_max, lat_max],
    resolution=0.01,
    units="degrees"
)

lons, lats = area.get_lonlats()
lons = lons[0]
lats = lats[:, 0]

with xr.open_dataset("/home/simonpf/data/etopo/globe_1.nc") as data:
    elev = data.interp(latitude=lats, longitude=lons)

In [ ]:
cpcir_data = cpcir_data.interp(latitude=lats, longitude=lons)

In [ ]:
cpcir_data["elevation"] = (("latitude", "longitude"), elev.elevation.data)

In [ ]:
from PIL import Image
bm = np.array(Image.open("/home/simonpf/data/natural_earth/blue_marble_sep.jpg"))
bm_area = create_area_def(
    "Blue marble",
    {"proj": "longlat", "datum": "WGS84"},
    area_extent=[-180, -90, 180, 90],
    shape=bm.shape[:2]
)

In [ ]:
from pyresample.kd_tree import resample_nearest
bm_r = resample_nearest(
    bm_area,
    bm,
    area,
    radius_of_influence=10e3
)

In [ ]:
cpcir_data["texture"] = (("latitude", "longitude", "channels"), bm_r)
plt.imshow(bm_r)

## Add contours to background

In [ ]:
import cmocean
from matplotlib.colors import BoundaryNorm, NoNorm, ListedColormap
from matplotlib import colormaps
from matplotlib.cm import ScalarMappable

cmap = colormaps["cmo.dense"]
colors = cmap(np.linspace(0, 1, 22))
colors[0, -1] = 0.0
norm = BoundaryNorm(np.logspace(-2, 1, 21), 21)
cinds = norm(cpcir_data.tiwp.data[0])

tiwp_rgb = colors[cinds.ravel()].reshape(cinds.shape + (4,))
mask = cinds > 0
bm_cntrs = np.concatenate((bm_r, 255 * np.ones(bm_r[..., [0]].shape, dtype="uint8")), -1)
bm_cntrs[mask] = 255 * tiwp_rgb[mask]

In [ ]:
plt.pcolormesh(bm_cntrs)

## Plotting functionality

In [ ]:
import pyvista as pv
#pv.start_xvfb()
#pv.global_theme.trame.server_proxy_enabled = True
#pv.global_theme.trame.server_proxy_prefix = "/proxy/"

In [ ]:
import cmocean
from scipy.signal import convolve
from matplotlib.cm import get_cmap, ScalarMappable
from matplotlib.colors import BoundaryNorm, LogNorm
import pyvista as pv

LEVELS = np.logspace(-3, np.log10(1), 11)
#LEVELS = np.linspace(0, 1, 11)[1:]
CMAP = get_cmap("cmo.dense").copy()
CMAP.set_under("#FFFFFF88")
NORM = BoundaryNorm(LEVELS, CMAP.N)
MAPPABLE = ScalarMappable(norm=NORM, cmap=CMAP)
lt = pv.LookupTable(
    "cmo.dense",
    scalar_range=(1e-3, 1e0),
    log_scale=True
)


def iwc_contours(data):
    x = data.longitude.data.astype(np.float32)
    y = data.latitude.data.astype(np.float32)
    
    elev = data.elevation.interp(
        latitude=data.latitude,
        longitude=data.longitude,
        method="nearest",
        kwargs={"fill_value": "extrapolate"}
    ).data
    
    k = np.ones((8, 8))
    elev_sum = convolve(elev, k, "same")
    elev_cts = convolve(np.ones_like(elev), k, "same")
    elev = elev_sum / elev_cts
    
    z = elev[:, :, None] + data.altitude.data[None, None, :]
    
    x, y, _ = np.meshgrid(x, y, data.altitude.data / 1e3)
    grid = pv.StructuredGrid(x, y, z / 1e3)
    grid["IWC [g / m^3]"] = data.tiwc.data.flatten(order="f")
    
    cts = grid.contour(LEVELS)
    cts["iwc_opc"] = LogNorm(1e-3, 1e-1)(cts["IWC [g / m^3]"], clip=True)
    colors = MAPPABLE.to_rgba(LEVELS)
    colors[:, -1] = np.linspace(0.1, 1.0, colors.shape[0])
    return cts, colors
    

In [ ]:
def iwc_volume(data):
    
    x = data.longitude.data.astype(np.float32)
    y = data.latitude.data.astype(np.float32)
    z = data.altitude.data.astype(np.float32) / 1e3
    dx = x[1] - x[0]
    dy = y[0] - y[1]
    dz = z[1] - z[0]
    
    ug = pv.UniformGrid(
        dimensions=(x.size, y.size, z.size),
        spacing=(dx, dy, dz),
        origin=(x[0], y[-1], z[0])
    )
    print(x[0], y[-1], z[0])   
    tiwc = np.transpose(np.flip(data.tiwc.data[0], 0), [1, 0, 2])
    tiwc = np.log10(np.maximum(tiwc, 1e-4))
    print(tiwc.size, x.size * y.size * z.size)
    ug["TIWC [kg / m^3]"] = tiwc.flatten(order="f")
    return ug


In [ ]:
def surface(data):
    lats = data.latitude.data
    lons = data.longitude.data
    z = data.elevation.data / 1e3
    x, y = np.meshgrid(lons, lats)
    grid = pv.StructuredGrid(x, y, z)
    grid.texture_map_to_plane(
        inplace=True,
        origin=[lon_min, lat_min, 0],
        point_u=[lon_max, lat_min, 0],
        point_v=[lon_min, lat_max, 0],
    )
    txt = pv.numpy_to_texture(np.copy(data.texture.data, order="f"))
    return grid, txt

In [ ]:
def setup_camera(scene):
    scene.camera_position = 'xz'
    scene.camera.elevation = 25
    scene.camera.zoom(1.5)
    scene.azimuth = 0

In [ ]:
from pansat.time import to_datetime64
iwc, colors = iwc_contours(cpcir_data[{"time": 0}].compute())

In [ ]:
from pansat.time import to_datetime64
iwc, colors = iwc_contours(cpcir_data[{"time": 0, "latitude": slice(0, 375 * 2)}])

In [ ]:
iwc, colors = iwc_contours(cpcir_data[{"time": 0}])

In [ ]:
cpcir_data["texture"] = (("latitude", "longitude", "channels"), bm_r)

In [ ]:
sfc, txt = surface(cpcir_data)

In [ ]:
pv.set_plot_theme("document")
pv.set_jupyter_backend('static')

In [ ]:
from matplotlib.colors import to_hex
scene = pv.Plotter(multi_samples=128, window_size=(1200, 600), line_smoothing=True, polygon_smoothing=True, point_smoothing=True)
scene.set_background("white")
scene.add_mesh(sfc, texture=txt);
#iwc_mesh = scene.add_mesh(iwc_clipped, scalars="IWC [g / m^3]", cmap=[to_hex(c) for c in colors], ambient=0.2, opacity="opacity", use_transparency=True);

In [ ]:
scene.enable_depth_peeling()

In [ ]:
scene.show_bounds(
    grid='back',
    location='outer',
    ticks='both',
    xlabel='Longitude [deg W]',
    ylabel='Latitude [deg N]',
    zlabel='Elevation [km]',
    font_size=8,
    axes_ranges=[lon_min, lon_max, lat_min, lat_max, 0, 20],
    bold=False
)

In [ ]:
scene.enable_depth_peeling()

In [ ]:
lat_t = lats[375 * 2]

In [ ]:
scene.add_lines(
    np.array([
        [lon_min, lat_t, 0.1],
        [lon_max, lat_t, 0.1]
    ]),
    color="grey"
)
scene.add_lines(
    np.array([
        [lon_min, lat_t, 20],
        [lon_max, lat_t, 20]
    ]),
    color="grey"
    
);

In [ ]:
from matplotlib.colors import to_hex
sargs = {
    "vertical": True,
    "position_x": 0.9,
    "position_y": 0.3
}
mesh = scene.add_mesh(
    iwc.copy(),
    scalars="IWC [g / m^3]",
    opacity="iwc_opc",
    cmap="dense",
    ambient=0.2,
    specular=0.5,
    scalar_bar_args=sargs
)

In [ ]:
scene.set_scale(zscale=0.2)

In [ ]:
setup_camera(scene)

In [ ]:
scene.remove_actor(mesh)

In [ ]:
scene.show()

In [ ]:
def rotate(scene, angle):
    scene.camera_position = 'xz'
    scene.camera.elevation = 25
    scene.camera.zoom(1.5)
    scene.camera.azimuth = angle

In [ ]:
scene.open_movie("nanmadol_rotation_2.mp4")

In [ ]:
for ang in np.linspace(-90, 90, 360):
    rotate(scene, ang)
    scene.write_frame()
    print(ang)

In [ ]:
for ang in np.linspace(90, -90, 360):
    rotate(scene, ang)
    scene.write_frame()
    print(ang)

In [ ]:
start_time = np.datetime64("2022-09-17T00:15:00")
end_time = np.datetime64("2022-09-17T12:00:00")
time_steps = np.arange(start_time, end_time, np.timedelta64(180, "s"))

In [ ]:
for time_step in time_steps:
    scene.remove_actor(mesh)
    iwc, colors = iwc_contours(cpcir_data.interp(time=time_step).compute())
    mesh = scene.add_mesh(
        iwc.copy(),
        scalars="IWC [g / m^3]",
        opacity="iwc_opc",
        cmap="dense",
        ambient=0.2,
        specular=0.5,
        scalar_bar_args=sargs
    )
    scene.write_frame()
    print(time_step)

In [ ]:
scene.close()